In [1]:
import rasterio
import numpy as np
import random
import time
from numba import njit
from numba.typed import Dict
from utils import *
from simulation import *
import logging

### Inputs & Parameters

In [4]:
# Date
date_ini = '1994'
date_fin = '2017'

# Paths of Input files
urban_path_ini = 'Inputs/urban1994_roads.gif'
urban_path_fin = 'Inputs/urban1994_roads.gif'
path_roads = 'Inputs/roads_94_mod.tif'
excluded_areas_path = 'Inputs/excluded.gif'
outside_boundaries_path = 'Inputs/outside_boundaries.tif'

# Objectifs Urbanization
objectif_urba = 1000
edge_growth,spont_growth,spread_growth,road_growth = 86,7,6,1

### Reading inputs and estimating new urbanisation


In [5]:
# Urban Areas
urban_ini = rasterio.open(urban_path_ini).read(1)
urban_fin = rasterio.open(urban_path_fin).read(1)

diff_urb = urban_fin - urban_ini
new_urb_index = np.where(diff_urb==255)
new_urb_index = [(a,b) for a,b in zip(new_urb_index[0],new_urb_index[1])]
new_urbanisation = len(new_urb_index)

# Roads
roads = rasterio.open(path_roads).read(1)

# Excluded Areas
excluded_areas = rasterio.open(excluded_areas_path).read(1)
excluded_areas_index = np.where(excluded_areas==255)
excluded_areas_index = [(a,b) for a,b in zip(excluded_areas_index[0],excluded_areas_index[1])]

x,y = excluded_areas.shape
index_uni = list(range(x*y))
index_double = [(a,b) for a in range(x) for  b in range(y)]

dict_double_uni = dict(zip(index_double,index_uni))
dict_uni_double = dict(zip(index_uni,index_double))

index_exclu_1d = [dict_double_uni[x] for x in excluded_areas_index]
index_exclu_1d_arr = np.array(index_exclu_1d)

# Outside Boundaries

raster = rasterio.open(outside_boundaries_path).read(1)
outside_boundaries = np.where(raster==0)
outside_boundaries_index = [(outside_boundaries[0][i],outside_boundaries[1][i]) for i in range(len(outside_boundaries[0]))]

# Raster Profile
raster_profile = rasterio.open(urban_path_ini).profile

# Ecluded + Outside Areas

exclu_outside_index = excluded_areas_index.copy()
exclu_outside_index.extend(outside_boundaries_index)

### Candidate Cells for Urbanization

In [6]:
edge_growth_cells,spread_growth_cells = create_edge_spread_poll(urban_ini,255,1)
edge_growth_cells = list(set(edge_growth_cells) - set(exclu_outside_index))
spread_growth_cells = list(set(spread_growth_cells) - set(exclu_outside_index))

road_growth_cells = create_road_poll(roads,urban_ini,255,exclu_outside_index,edge_growth_cells,spread_growth_cells)
spont_growth_cells = create_spont_poll(urban_ini,0,edge_growth_cells,road_growth_cells,spread_growth_cells,exclu_outside_index)

edge_growth_cells_da = tuple_to_double_arr(edge_growth_cells)
spread_growth_cells_da = tuple_to_double_arr(spread_growth_cells)
road_growth_cells_da = tuple_to_double_arr(road_growth_cells)
spont_growth_cells_da = tuple_to_double_arr(spont_growth_cells)

### Create Order of Urbanization

In [76]:
objectif_urba = 10
nb_cells_edge = int(objectif_urba * edge_growth / 100)
nb_cells_road = int(objectif_urba * road_growth / 100)
nb_cells_spread = int(objectif_urba * spread_growth / 100)
nb_cells_spont = int(objectif_urba * spont_growth / 100)

order_urbanization = np.ones(nb_cells_edge)
order_urbanization = np.append(order_urbanization, 2 * np.ones(nb_cells_spread))
order_urbanization = np.append(order_urbanization, 3 * np.ones(nb_cells_road))
order_urbanization = np.append(order_urbanization, 4 * np.ones(nb_cells_spont))
order_urbanization = order_urbanization.astype('int8')
np.random.shuffle(order_urbanization)

In [147]:
evo_candidate_cells = urban_ini.copy()
evo_candidate_cells[edge_growth_cells_da] = 1
evo_candidate_cells[spread_growth_cells_da] = 2
evo_candidate_cells[road_growth_cells_da] = 3
evo_candidate_cells[spont_growth_cells_da] = 4

initial_urba = np.where(urban_ini==255)
initial_urba = [(initial_urba[0][i],initial_urba[1][i]) for i in range(len(initial_urba[0]))]
initial_urba = np.array(initial_urba)

In [191]:
[6,195] in initial_urba

True

In [204]:
%timeit simulation_numba_test(evo_candidate_cells,initial_urba,order_urbanization,1,index_exclu_1d_arr)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of Function(<built-in function contains>) with argument(s) of type(s): (array(int64, 2d, C), list(int64))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 3:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 4:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 5:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 6:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 7:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 8:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 9:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 10

In [193]:
dist = 1
possib = [[a,b]  for a in range(x-dist,x+dist+1) for b in range(y-dist,y+dist+1) if (a,b)!=(x,y)]

In [198]:
[a for a in possib if a in initial_urba]

[]

In [194]:
possib

[[1253, 1547],
 [1253, 1548],
 [1253, 1549],
 [1254, 1547],
 [1254, 1549],
 [1255, 1547],
 [1255, 1548],
 [1255, 1549]]

In [203]:
@njit
def simulation_numba_test(urb_fin_new,ini_urb,order_urbanization,dist,index_exclu_1d_arr):
    new_urb = np.empty(0)
    lig,col = urb_fin_new.shape[0],urb_fin_new.shape[1]

    for rand in order_urbanization:
        growth_cells = np.where(urb_fin_new==rand)
        rand_index = random.randint(0,len(growth_cells[0])-1)
        x,y = growth_cells[0][rand_index],growth_cells[1][rand_index]
        new_urb = np.append(new_urb,[(x,y)])
        tmp = np.append(ini_urb,np.array([(x,y)]).reshape(1,2)).copy()
        urb_fin_new[x,y] = 255
        adj_cells = urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1].copy().reshape((2*dist+1)**2)

        if rand==1:
            new_val =  np.array(list(map(lambda x: 1 if x not in [0,255] else x , adj_cells))).reshape(2*dist+1,2*dist+1)
            urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1] = new_val

        elif rand==2:
            new_val =  np.array(list(map(lambda x: 1 if x not in [0,255] else x , adj_cells))).reshape(2*dist+1,2*dist+1)
            urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1] = new_val

            possib = [[a,b]  for a in range(x-dist,x+dist+1) for b in range(y-dist,y+dist+1) if (a,b)!=(x,y)]
            
            x = [a for a in possib if a in ini_urb][0]

            adj_cells_old = urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1].copy().reshape((2*dist+1)**2)          
            new_val =  np.array(list(map(lambda x: 1 if x not in [0,255] else x , adj_cells_old))).reshape(2*dist+1,2*dist+1)
            urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1] = new_val

        elif rand==3:
            if np.sum(adj_cells)==255:
                adj_cells = urb_fin_new[x-1:x+2,y-1:y+2].copy().reshape(9)
                new_val =  np.array(list(map(lambda x: 2 if x not in [0,3,255] else x , adj_cells))).reshape(3,3)
                urb_fin_new[x-1:x+2,y-1:y+2] = new_val

        else:
            adj_cells = urb_fin_new[x-1:x+2,y-1:y+2].copy().reshape(9)
            new_val =  np.array(list(map(lambda x: 2 if x not in [0,3,255] else x , adj_cells))).reshape(3,3)
            urb_fin_new[x-1:x+2,y-1:y+2] = new_val 

        urb_fin_new_bis = urb_fin_new.copy().reshape(lig*col)
        urb_fin_new_bis[index_exclu_1d_arr] = 0
        urb_fin_new = urb_fin_new_bis.reshape((lig,col)).copy()
        
    return new_urb

In [70]:
@njit
def test(urb_fin_new,urban_ini,order_urbanization,dist,index_exclu_1d_arr):
    new_urb = np.empty(0)
    growth_cells = np.where(urb_fin_new==1)
    rand_index = random.randint(0,len(growth_cells[0])-1)
    x,y = growth_cells[0][rand_index],growth_cells[1][rand_index]
    new_urb = np.append(new_urb,[(x,y)])
    urban_ini = np.append(urban_ini,[(x,y)])
    urb_fin_new[x,y] = 255
    adj_cells = urb_fin_new[x-dist:x+dist+1,y-dist:y+dist+1].copy().reshape((2*dist+1)**2)

In [71]:
%timeit test(evo_candidate_cells,np.array(initial_urba),order_urbanization,1,index_exclu_1d_arr)

110 ms ± 6.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
urb_fin = urban_ini.copy()
ea,eb = [a[0] for a in new_urban[1]],[a[1] for a in new_urban[1]]
urb_fin[ea,eb] = 10

ea,eb = [a[0] for a in new_urban[2]],[a[1] for a in new_urban[2]]
urb_fin[ea,eb] = 20

ea,eb = [a[0] for a in new_urban[3]],[a[1] for a in new_urban[3]]
urb_fin[ea,eb] = 30

ea,eb = [a[0] for a in new_urban[4]],[a[1] for a in new_urban[4]]
urb_fin[ea,eb] = 40

In [84]:
out = 'Test_23000.gif'
with rasterio.open(out,'w', **raster_profile) as dst:
    dst.write(urb_fin.reshape(1,1254,1548))

In [2]:
logging.basicConfig(filename='log_objectif.log',level=logging.INFO,
    format='%(asctime)s,%(msecs)d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s')